In [99]:
import json
import itertools


In [100]:
current_config = "3_scenarios_config"
shared_scenarios_name = "3_shared_scenarios"

when_expression = "takeoff == 1"

In [101]:
file_path = f'res/scenario_configs/{current_config}.json'
with open(file_path, 'r', encoding='utf-8') as file:
    scenarios_config = json.load(file)

In [102]:
def generate_all_combinations(given_or_then_config):
    # Gerar todas as combinações possíveis de cada variável individual
    var_combinations_dict = given_or_then_config["var_combinations"]
    is_changeable_number_variables = given_or_then_config["is_changeable_number_variables"]
    extra_expression_list = given_or_then_config["extra_expressions"]

    amount_vars = len(var_combinations_dict) + len(extra_expression_list)

    all_combinations_all_vars_list = []

    for var, expressions in var_combinations_dict.items():
        all_combinations_all_vars_list.append(expressions)

    if len(extra_expression_list) > 0:
        for expr in extra_expression_list:
            all_combinations_all_vars_list.append([expr])

    
    final_combinations = []

    if is_changeable_number_variables:
        # Combinar de 1 até o número total de variáveis
        for i in range(1, amount_vars + 1):
            for subset in itertools.combinations(all_combinations_all_vars_list, i):
                # Gerar o produto cartesiano dentro de cada subconjunto
                final_combinations.extend(itertools.product(*subset))
    else:
        # Se a quantidade for fixa, simplesmente gerar o produto cartesiano de todas
        final_combinations = list(itertools.product(*all_combinations_all_vars_list))

    return final_combinations





In [103]:
given_config = scenarios_config['Given']
then_config = scenarios_config['Then']

given_combinations = generate_all_combinations(given_config)
then_combinations = generate_all_combinations(then_config)

final_combinations = list(itertools.product(given_combinations, then_combinations))
final_combinations

[(('battery >= 10',), ('height >= 30',)),
 (('battery >= 10',), ('source_distance == 0',)),
 (('battery >= 10',), ('a > 10',)),
 (('battery >= 10',), ('b > 10',)),
 (('battery >= 10',), ('height >= 30', 'source_distance == 0')),
 (('battery >= 10',), ('height >= 30', 'a > 10')),
 (('battery >= 10',), ('height >= 30', 'b > 10')),
 (('battery >= 10',), ('source_distance == 0', 'a > 10')),
 (('battery >= 10',), ('source_distance == 0', 'b > 10')),
 (('battery >= 10',), ('a > 10', 'b > 10')),
 (('battery >= 10',), ('height >= 30', 'source_distance == 0', 'a > 10')),
 (('battery >= 10',), ('height >= 30', 'source_distance == 0', 'b > 10')),
 (('battery >= 10',), ('height >= 30', 'a > 10', 'b > 10')),
 (('battery >= 10',), ('source_distance == 0', 'a > 10', 'b > 10')),
 (('battery >= 10',),
  ('height >= 30', 'source_distance == 0', 'a > 10', 'b > 10')),
 (('satellite_count >= 7',), ('height >= 30',)),
 (('satellite_count >= 7',), ('source_distance == 0',)),
 (('satellite_count >= 7',), ('a 

In [104]:
def generate_candidates(final_combinations, _when_expression):
    candidates = []
    for idx, (given, then) in enumerate(final_combinations, start=1):
        given_part = " AND ".join(given)
        then_part = " AND ".join(then)

        candidate = {
            f"Candidate_{idx}": {
                "Given": given_part,
                "When": f"{when_expression}",  # Exemplo fixo de 'When'
                "Do": f"behavior_{idx}",  # Exemplo fixo de 'Do'
                "Then": then_part
            }
        }

        candidates.append(candidate)
        
    return candidates


In [105]:
candidaties_json=generate_candidates(final_combinations, _when_expression=when_expression)

file_name = f"res/outcome/shared_scenarios/{shared_scenarios_name}.json"
with open(file_name, 'w') as json_file:
        json.dump(candidaties_json, json_file, indent=2)